# TPOTやってみた
### what's TPOT
PythonのAuto Machine Learningツール。<br>
遺伝アルゴリズムを使用して最適化した機械学習のパイプラインを作成してくれる。<br>
Scikit-learnを元に作成されたライブラリなので、Scikit-learnに慣れている方はサクッと使える。<br>


### bostonデータセットで回帰問題

In [3]:
from sklearn.datasets import load_boston

data = load_boston()

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.data,
                                                    data.target,
                                                   train_size=0.7,
                                                   test_size=0.3,
                                                   random_state=1)

#### tpotインスタンスの引数
* **generations**<br>
遺伝アルゴリズムを何代目まで計算するか
* **population_size**<br>

* **vervosity**<br>
処理を実行した場合に、どのようなログを出力するか？
* **n_jobs**<br>
並列処理を行うか
* **random_state**<br>
乱数のシード値。再現性を出す。
* **memory**<br>
ざっくりいえば計算コストを下げるか否か。

In [5]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5,
                     population_size=100,
                     verbosity=2)

In [6]:
tpot.fit(x_train, y_train)

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -12.721885532428692
Generation 2 - Current best internal CV score: -12.721885532428692
Generation 3 - Current best internal CV score: -12.721885532428692
Generation 4 - Current best internal CV score: -11.446755722175743
Generation 5 - Current best internal CV score: -11.446755722175743

Best pipeline: GradientBoostingRegressor(SelectFwe(input_matrix, alpha=0.015), alpha=0.99, learning_rate=0.1, loss=huber, max_depth=7, max_features=0.9500000000000001, min_samples_leaf=9, min_samples_split=6, n_estimators=100, subsample=0.8)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [9]:
tpot.score(x_test, y_test)

-8.617050563406771

In [10]:
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('selectfwe',
                 SelectFwe(alpha=0.015,
                           score_func=<function f_regression at 0x7f22385e3ea0>)),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(alpha=0.99, criterion='friedman_mse',
                                           init=None, learning_rate=0.1,
                                           loss='huber', max_depth=7,
                                           max_features=0.9500000000000001,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=9,
                                           min_samples_split=6,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100,
                

In [11]:
tpot.evaluated_individuals_

{'KNeighborsRegressor(SelectPercentile(input_matrix, SelectPercentile__percentile=25), KNeighborsRegressor__n_neighbors=44, KNeighborsRegressor__p=2, KNeighborsRegressor__weights=uniform)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': -28.73970066639506},
 'ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=0.15000000000000002, ElasticNetCV__tol=0.01)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 1,
  'internal_cv_score': -32.45765037921385},
 'ExtraTreesRegressor(DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=2, DecisionTreeRegressor__min_samples_leaf=2, DecisionTreeRegressor__min_samples_split=14), ExtraTreesRegressor__bootstrap=False, ExtraTreesRegressor__max_features=0.35000000000000003, ExtraTreesRegressor__min_samples_leaf=9, ExtraTreesRegressor__min_samples_split=7, ExtraTreesRegressor__n_estimator

In [12]:
tpot.export("bostn_tpot.py")

In [14]:
! cat bostn_tpot.py

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFwe, f_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=None)

# Average CV score on the training set was:-11.446755722175743
exported_pipeline = make_pipeline(
    SelectFwe(score_func=f_regression, alpha=0.015),
    GradientBoostingRegressor(alpha=0.99, learning_rate=0.1, loss="huber", max_depth=7, max_features=0.9500000000000001, min_samples_leaf=9, min_samples_split=6, n_estimators=100, subsample=0.8)
)

exported_pipeline.fit(training

### irisデータセットで分類問題

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data,
                                                   iris.target,
                                                   train_size = 0.7,
                                                   test_size = 0.3,
                                                   random_state=1)

In [29]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5,
                     verbosity=2,
                     memory="auto",
                     n_jobs=4)

In [30]:
print(y_test)

[0 1 1 0 2 1 2 0 0 2 1 0 2 1 1 0 1 1 0 0 1 1 1 0 2 1 0 0 1 2 1 2 1 2 2 0 1
 0 1 2 2 0 2 2 1]


In [31]:
tpot.fit(x_train, y_train)

Generation 1 - Current best internal CV score: 0.9709090909090909
Generation 2 - Current best internal CV score: 0.9809090909090908
Generation 3 - Current best internal CV score: 0.9809090909090908
Generation 4 - Current best internal CV score: 0.9809090909090908
Generation 5 - Current best internal CV score: 0.9809090909090908

Best pipeline: GaussianNB(CombineDFs(MultinomialNB(input_matrix, alpha=0.1, fit_prior=True), input_matrix))


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory='auto',
               mutation_rate=0.9, n_jobs=4, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [33]:
tpot.export("iris_tpot.py")

In [35]:
! cat iris_tpot.py

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=None)

# Average CV score on the training set was:0.9809090909090908
exported_pipeline = make_pipeline(
    make_union(
        StackingEstimator(estimator=MultinomialNB(alpha=0.1, fit_prior=True)),
        FunctionTransformer(copy)
    ),
    GaussianNB()
)

exported_pipeline.fit(training_features, training_tar